In [1]:
# Import required libraries
import os
import torch
import shutil
from ultralytics import YOLO
from pathlib import Path
from PIL import Image

# Display system info
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
    print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
print(f"\nWorking directory: {os.getcwd()}")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/home/vscode/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
PyTorch version: 2.8.0+cu129
CUDA available: True
CUDA device: NVIDIA GeForce RTX 3050 Laptop GPU
GPU memory: 4.29 GB

Working directory: /workspace


## Step 1: Prepare Combined Dataset

We'll create a unified dataset structure combining CORD and SROIE2019 data.
For unified text detection, we'll use a single class (0: text).

In [2]:
# Define paths
COMBINED_DIR = Path('data/combined_dataset')
CORD_RAW_DIR = Path('data/cord/raw')
SROIE_DIR = Path('data/SROIE2019')

# Create combined directory structure
for split in ['train', 'val', 'test']:
    (COMBINED_DIR / split).mkdir(parents=True, exist_ok=True)

print(f"✓ Created combined dataset directory: {COMBINED_DIR}")

✓ Created combined dataset directory: data/combined_dataset


In [3]:
# Function to convert multi-class labels to single class (text = 0)
def convert_to_single_class(input_file: Path, output_file: Path):
    """Convert any YOLO label file to single class (0)."""
    with open(input_file, 'r') as f:
        lines = f.readlines()
    
    converted_lines = []
    for line in lines:
        parts = line.strip().split()
        if len(parts) >= 5:
            # Replace class ID with 0, keep bbox coordinates
            converted_lines.append(f"0 {' '.join(parts[1:])}\n")
    
    with open(output_file, 'w') as f:
        f.writelines(converted_lines)
    
    return len(converted_lines)

print("✓ Defined label conversion function")

✓ Defined label conversion function


In [4]:
# Copy and convert CORD data
print("="*70)
print("Processing CORD Dataset")
print("="*70)

cord_splits = {
    'train': CORD_RAW_DIR / 'train',
    'val': CORD_RAW_DIR / 'validation',
    'test': CORD_RAW_DIR / 'test'
}

cord_stats = {'train': 0, 'val': 0, 'test': 0}

for split, src_dir in cord_splits.items():
    if not src_dir.exists():
        print(f"Warning: {src_dir} does not exist")
        continue
    
    dest_dir = COMBINED_DIR / split
    
    # Copy images
    for img_file in list(src_dir.glob('*.png')) + list(src_dir.glob('*.jpg')):
        shutil.copy2(img_file, dest_dir / img_file.name)
    
    # Convert labels to single class
    for label_file in src_dir.glob('*.txt'):
        convert_to_single_class(label_file, dest_dir / label_file.name)
        cord_stats[split] += 1

print(f"✓ CORD - Train: {cord_stats['train']}, Val: {cord_stats['val']}, Test: {cord_stats['test']}")

Processing CORD Dataset
✓ CORD - Train: 800, Val: 100, Test: 100


In [5]:
# Process SROIE2019 data using the converter utility
print("="*70)
print("Processing SROIE2019 Dataset")
print("="*70)

import sys
sys.path.insert(0, '/workspace/src')
from detection.sroie_to_yolo import SROIEToYOLOConverter

converter = SROIEToYOLOConverter()

# Process SROIE training data -> combined train
sroie_train_box = SROIE_DIR / 'train' / 'box'
sroie_train_img = SROIE_DIR / 'train' / 'img'

sroie_train_count = 0
if sroie_train_box.exists():
    for box_file in sroie_train_box.glob('*.txt'):
        # Convert box file to YOLO format
        output_label = COMBINED_DIR / 'train' / box_file.name
        success = converter.convert_txt_to_yolo(str(box_file), str(output_label))
        if success:
            # Copy corresponding image
            img_name = box_file.stem + '.jpg'
            img_path = sroie_train_img / img_name
            if img_path.exists():
                shutil.copy2(img_path, COMBINED_DIR / 'train' / img_name)
                sroie_train_count += 1

print(f"✓ SROIE Train: {sroie_train_count} samples added to training set")

# Process SROIE test data -> combined test
sroie_test_box = SROIE_DIR / 'test' / 'box'
sroie_test_img = SROIE_DIR / 'test' / 'img'

sroie_test_count = 0
if sroie_test_box.exists():
    for box_file in sroie_test_box.glob('*.txt'):
        output_label = COMBINED_DIR / 'test' / box_file.name
        success = converter.convert_txt_to_yolo(str(box_file), str(output_label))
        if success:
            img_name = box_file.stem + '.jpg'
            img_path = sroie_test_img / img_name
            if img_path.exists():
                shutil.copy2(img_path, COMBINED_DIR / 'test' / img_name)
                sroie_test_count += 1

print(f"✓ SROIE Test: {sroie_test_count} samples added to test set")

Processing SROIE2019 Dataset
✓ SROIE Train: 626 samples added to training set
Error reading data/SROIE2019/test/box/X51006619503.txt: 'utf-8' codec can't decode byte 0xa3 in position 407: invalid start byte
✓ SROIE Test: 346 samples added to test set


In [6]:
# Verify combined dataset
print("="*70)
print("Combined Dataset Summary")
print("="*70)

for split in ['train', 'val', 'test']:
    split_dir = COMBINED_DIR / split
    images = list(split_dir.glob('*.png')) + list(split_dir.glob('*.jpg'))
    labels = list(split_dir.glob('*.txt'))
    print(f"{split.capitalize():6}: {len(images)} images, {len(labels)} labels")

print("="*70)

Combined Dataset Summary
Train : 1426 images, 1426 labels
Val   : 100 images, 100 labels
Test  : 446 images, 446 labels


In [7]:
# Create data.yaml for combined dataset
data_yaml_content = """path: /workspace/data/combined_dataset
train: train
val: val
test: test
nc: 1
names:
  0: text
"""

data_yaml_path = COMBINED_DIR / 'data.yaml'
with open(data_yaml_path, 'w') as f:
    f.write(data_yaml_content)

print(f"✓ Created data.yaml at {data_yaml_path}")
print("\nContents:")
print(data_yaml_content)

✓ Created data.yaml at data/combined_dataset/data.yaml

Contents:
path: /workspace/data/combined_dataset
train: train
val: val
test: test
nc: 1
names:
  0: text



## Step 2: Train YOLOv11n on Combined Dataset

Now we'll train the model from scratch on the combined dataset.

In [8]:
# Load pretrained YOLOv11n model
print("Loading YOLOv11n pretrained model...")
model = YOLO('yolo11n.pt')
print("✓ Model loaded successfully!")

Loading YOLOv11n pretrained model...
✓ Model loaded successfully!


In [10]:
# Train on combined dataset
print("="*70)
print("Starting YOLOv11n Training on Combined Dataset")
print("="*70)

data_yaml_path = 'data/combined_dataset/data.yaml'

results = model.train(
    data=data_yaml_path,
    epochs=50,
    imgsz=416,
    device=0,
    batch=8,
    patience=10,
    save=True,
    project='runs/yolo11n_combined',
    name='train',
    verbose=False,
    plots=True,
    seed=42,
)

print("\n" + "="*70)
print("Training Complete!")
print("="*70)

Starting YOLOv11n Training on Combined Dataset
New https://pypi.org/project/ultralytics/8.3.240 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.223 🚀 Python-3.11.13 torch-2.8.0+cu129 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data/combined_dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=416, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, m

Corrupt JPEG data: 33 extraneous bytes before marker 0xd9


       1/50     0.754G        2.4      2.091      1.382        248        416: 97% ━━━━━━━━━━━╸ 173/179 4.6it/s 44.9s<1.3ss

Corrupt JPEG data: 33 extraneous bytes before marker 0xd9


       1/50     0.754G      2.392      2.074      1.378         42        416: 100% ━━━━━━━━━━━━ 179/179 3.6it/s 49.4s0.3s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 7/7 1.1it/s 6.1s0.3ss
                   all        100       2186      0.659      0.665      0.672      0.367

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
       2/50      0.85G       2.03      1.388       1.22         42        416: 100% ━━━━━━━━━━━━ 179/179 4.7it/s 38.0s1.9ss
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 7/7 5.1it/s 1.4s0.2s
                   all        100       2186      0.782      0.787       0.82      0.481

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
       3/50     0.863G      1.697      1.185      1.106        247        416: 2% ──────────── 4/179 4.5it/s 0.9s<38.5s

...

      48/50      1.72G      1.261     0.8148      1.015         12        416: 100% ━━━━━━━━━━━━ 179/179 4.5it/s 39.6s0.2ss
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 7/7 4.6it/s 1.5s0.2s
                   all        100       2186      0.953        0.9      0.958      0.703

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      49/50      1.74G      1.299     0.8505      1.035        147        416: 42% ━━━━━─────── 76/179 13.1it/s 14.5s<7.8s

Corrupt JPEG data: 33 extraneous bytes before marker 0xd9


      49/50      1.74G      1.241     0.7951      1.008         41        416: 100% ━━━━━━━━━━━━ 179/179 4.7it/s 37.9s0.2ss
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 7/7 5.4it/s 1.3s0.2s
                   all        100       2186      0.946      0.905      0.959      0.704

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      50/50      1.75G      1.261     0.8263      1.019        132        416: 35% ━━━━──────── 63/179 5.3it/s 11.6s<22.0s

Corrupt JPEG data: 33 extraneous bytes before marker 0xd9


      50/50      1.75G      1.251     0.8112      1.015         28        416: 100% ━━━━━━━━━━━━ 179/179 5.5it/s 32.3s0.2ss
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 7/7 5.9it/s 1.2s0.2s
                   all        100       2186      0.942      0.905      0.958      0.701

50 epochs completed in 0.524 hours.
Optimizer stripped from /workspace/runs/yolo11n_combined/train/weights/last.pt, 5.4MB
Optimizer stripped from /workspace/runs/yolo11n_combined/train/weights/best.pt, 5.4MB

Validating /workspace/runs/yolo11n_combined/train/weights/best.pt...
Ultralytics 8.3.223 🚀 Python-3.11.13 torch-2.8.0+cu129 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 7/7 2.2it/s 3.1s0.3ss
                   all        100       2186      0.947   

In [11]:
# Training results summary
print(f"Best model path: {model.trainer.best}")
print(f"Results directory: {results.save_dir}")

if hasattr(results, 'box'):
    print(f"\nCombined Dataset Metrics:")
    print(f"  mAP50: {results.box.map50:.4f}")
    print(f"  mAP50-95: {results.box.map:.4f}")
    print(f"  Precision: {results.box.mp:.4f}")
    print(f"  Recall: {results.box.mr:.4f}")
    
    print(f"\nComparison with CORD-only Baseline:")
    print(f"  {'Metric':<12} {'CORD-only':>12} {'Combined':>12} {'Change':>12}")
    print(f"  {'-'*48}")
    baseline = {'mAP50': 0.4203, 'mAP50-95': 0.2938, 'Precision': 0.5951, 'Recall': 0.4255}
    current = {'mAP50': results.box.map50, 'mAP50-95': results.box.map, 
               'Precision': results.box.mp, 'Recall': results.box.mr}
    for metric in baseline:
        change = current[metric] - baseline[metric]
        sign = '+' if change >= 0 else ''
        print(f"  {metric:<12} {baseline[metric]:>12.4f} {current[metric]:>12.4f} {sign}{change:>11.4f}")

Best model path: /workspace/runs/yolo11n_combined/train/weights/best.pt
Results directory: /workspace/runs/yolo11n_combined/train

Combined Dataset Metrics:
  mAP50: 0.9585
  mAP50-95: 0.7028
  Precision: 0.9474
  Recall: 0.9026

Comparison with CORD-only Baseline:
  Metric          CORD-only     Combined       Change
  ------------------------------------------------
  mAP50              0.4203       0.9585 +     0.5382
  mAP50-95           0.2938       0.7028 +     0.4090
  Precision          0.5951       0.9474 +     0.3523
  Recall             0.4255       0.9026 +     0.4771


## Step 3: Validate on Test Set

In [12]:
# Load best model and validate
best_model_path = 'runs/yolo11n_combined/train/weights/best.pt'
best_model = YOLO(best_model_path)

print("Running validation on test set...")
val_results = best_model.val(
    data='data/combined_dataset/data.yaml',
    device=0,
    batch=4,
    imgsz=416,
    workers=0,
)

print(f"\nTest Set Performance:")
if hasattr(val_results, 'box'):
    print(f"  mAP50: {val_results.box.map50:.4f}")
    print(f"  mAP50-95: {val_results.box.map:.4f}")
    print(f"  Precision: {val_results.box.mp:.4f}")
    print(f"  Recall: {val_results.box.mr:.4f}")

Running validation on test set...
Ultralytics 8.3.223 🚀 Python-3.11.13 torch-2.8.0+cu129 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access ✅ (ping: 2.5±0.0 ms, read: 80.6±39.5 MB/s, size: 1084.5 KB)
val: Scanning /workspace/data/combined_dataset/val.cache... 100 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 100/100 102.6Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 25/25 2.3it/s 10.7s0.6s
                   all        100       2186      0.947      0.901      0.958      0.708
Speed: 1.0ms preprocess, 17.1ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to /workspace/runs/detect/val

Test Set Performance:
  mAP50: 0.9579
  mAP50-95: 0.7081
  Precision: 0.9471
  Recall: 0.9012


## Step 4: Run Inference on Sample Images

In [ ]:
# Run predictions on test set
best_model_path = 'runs/yolo11n_combined/train/weights/best.pt'
best_model = YOLO(best_model_path)

print("Running inference on test set...")
pred_results = best_model.predict(
    source='data/combined_dataset/test',
    conf=0.25,
    device=0,
    batch=4,
    imgsz=416,
    workers=0,
    save=True,
    project='runs/yolo11n_combined',
    name='predictions',
)

print(f"\n✓ Predictions saved to: runs/yolo11n_combined/predictions")
print(f"✓ Total images processed: {len(pred_results)}")

## Step 5: Export Best Model

In [ ]:
# Export best model for deployment
best_model_path = 'runs/yolo11n_combined/train/weights/best.pt'
best_model = YOLO(best_model_path)

# Copy best model to a convenient location
import shutil
final_model_path = Path('models')
final_model_path.mkdir(exist_ok=True)
shutil.copy2(best_model_path, final_model_path / 'yolo11n_receipt_detector.pt')

print(f"✓ Best model saved to: {final_model_path / 'yolo11n_receipt_detector.pt'}")
print(f"\nModel can be loaded with:")
print(f"  model = YOLO('models/yolo11n_receipt_detector.pt')")